In [2]:
import Downloader
import Reddit_Sql
import Subreddit_lists
import praw
import Api_ID


Successfully connected


Version 7.4.0 of praw is outdated. Version 7.6.0 was released Tuesday May 10, 2022.


In [ ]:

reddit = praw.Reddit(
    client_id=Api_ID.App_id,
    client_secret=Api_ID.Secret,
    password="Zuccjuicer",
    user_agent="PRAW:Link Grabbing bot by u/MetaNotFacebook",
    username="MetaNotFacebook",
)